In [1]:
# Store data with pickle
# https://docs.python.org/3/library/pickle.html

In [2]:
import pickle
import pandas as pd
import sqlalchemy

In [3]:
# save a file in the format pickle

In [4]:
data = [{"name": "mack", "last_name": "nilan"}]
with open("users.pkl", "wb") as f:
    pickle.dump(data, f)  # pickle.dump

In [5]:
# read a file in the format pickle and know the type of file

In [6]:
data_from_pickle = pickle.load(open("users.pkl", "rb"))
print(data_from_pickle, type(data_from_pickle))

[{'name': 'mack', 'last_name': 'nilan'}] <class 'list'>


In [7]:
df = pd.DataFrame(data)
df.head()

name last_name
0  mack     nilan

In [8]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html
# creates a files -> data_frame_pickle
df.to_pickle("data_frame_pickle")
df.dtypes

name         object
last_name    object
dtype: object

In [9]:
df2 = pd.read_pickle("data_frame_pickle")
df2.head()

name last_name
0  mack     nilan

In [10]:
df2.dtypes

name         object
last_name    object
dtype: object

In [11]:
# DO THE SAME THING WITH A CSV FILE
df.to_csv("data_frame_pickle.csv", index=False)

# SQLAlchemy
#### SQLAlchmey & Django have a similar object-relational mappings (aka `ORM`) are ways to connect Python to a SQL database.

In [12]:
# https://www.sqlalchemy.org/

In [13]:
class Movie1:
    name = 'Unknown'
    genre = 'Action'
   
    def __init__(self, name='', *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name

In [14]:
movie_obj = Movie1(name="The Matrix", genre="Sci-Fi")
movie_obj.name

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [16]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str = 'Unknown'
    genre:str = 'Action'
    year:int = None

In [17]:
movie_obj = Movie(name="The Matrix", genre="Sci-Fi")
movie_obj.name
movie_obj.genre

'Sci-Fi'

In [18]:
data = [
    {
        "name": "Interstellar",
        "genre": "Sci-Fi"
    },
    {
        "name": "The Martian",
        "genre": "Sci-Fi"
    },
    {
        "name": "Arrival",
        "genre": "Sci-Fi"
    }
]

In [19]:
df = pd.DataFrame(data)
df.head()

name   genre
0  Interstellar  Sci-Fi
1   The Martian  Sci-Fi
2       Arrival  Sci-Fi

In [ ]:
# sqlalchemy
# https://docs.sqlalchemy.org/en/14/intro.html#installation
# https://docs.sqlalchemy.org/en/14/
# https://docs.sqlalchemy.org/en/14/tutorial/index.html
# 
# https://www.tutorialspoint.com/sqlalchemy/index.htm <-----

In [20]:
import sqlalchemy
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import sessionmaker

In [21]:
engine = create_engine("sqlite:///app_test.db")  # mysql, postgres

In [22]:
Session = sessionmaker(bind=engine)
my_sess = Session()

In [23]:
# SE DECLARA LA CLASE, Base
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [24]:
class Movie(Base): # table
    __tablename__ = "movies" # TABLE NAME
    
    id = Column(Integer, primary_key=True) # auto created for us
    name = Column(String) # 'Unknown'  # COLUMN
    genre = Column(String) # 'Action'  # COLUMN
    description = Column(String) # 'Action'  # COLUMN
    year = Column(Integer, nullable=True) # None  # COLUMN
    
    
    def __repr__(self):
        return f"<Movie name={self.name}>"

# saved instance = row

In [25]:
# ADD TABLE(S) TO DATABASE
Base.metadata.create_all(engine)

In [26]:
movie_obj = Movie(name='Interstellar', genre='Sci-Fi')
print(movie_obj.name)

Interstellar


In [27]:
my_sess.add(movie_obj) # prepare to save
my_sess.commit() # save

In [28]:
movie_obj.id

10

In [29]:
movie_obj2 = Movie(name='The Matrix', genre='Sci-Fi')
print(movie_obj2.name, movie_obj2.id)

The Matrix None


In [30]:
movie_obj3 = Movie(name='Lovecraft Country', genre='Sci-Fi')
print(movie_obj3.name, movie_obj3.id)

Lovecraft Country None


In [31]:
my_sess.add(movie_obj2) # PREPARE TO SAVE
my_sess.add(movie_obj3) # PREPARE TO SAVE
my_sess.commit() # SAVE

In [32]:
print(movie_obj2.id, movie_obj3.id)

11 12


## CRUD in SQLAlchemy
    - Create
    - Retrive
    - Update
    - Delete

In [33]:
# SE CREA OTRA SESION
Session = sessionmaker(bind=engine)
session = Session()

In [34]:
# CREATE
movie = Movie(name='Guaridans of the Galaxy')
session.add(movie)
session.commit()

In [35]:
# RETRIVE

# GET 1 ITEM
movie_a = session.query(Movie).get(2)
print(movie_a.id, movie_a.name, movie_a.description)

2 Matrix Revolutions Cool movie


In [36]:
# LIST
# qs -> Query Set
qs = session.query(Movie).all()
print(qs)

[<Movie name=Matrix Revolutions>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>, <Movie name=Interstellar>, <Movie name=Matrix Revolutions>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>, <Movie name=Interstellar>, <Movie name=Interstellar>, <Movie name=The Matrix>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>]


In [37]:
# LIST & FILTER BY COLUMN VALUE
qs = session.query(Movie).filter_by(name='Lovecraft Country').all()
print(qs)

[<Movie name=Lovecraft Country>, <Movie name=Lovecraft Country>, <Movie name=Lovecraft Country>]


In [38]:
# LIST & FILTER BY COLUMN VALUE CONTAINING SOMETHING
qs = session.query(Movie).filter(Movie.name.contains("Galaxy")).all()
qs

[<Movie name=Guaridans of the Galaxy>,
 <Movie name=Guaridans of the Galaxy>,
 <Movie name=Guaridans of the Galaxy>]

In [40]:
# LIST & FILTER BY COLUMN VALUE CONTAINING SOMETHING
my_query = input("What are you looking for?\n") or "Unknown"
qs = session.query(Movie).filter(Movie.name.contains(my_query)).all()
print(qs)

What are you looking for?
Galaxy
[<Movie name=Guaridans of the Galaxy>, <Movie name=Guaridans of the Galaxy>, <Movie name=Guaridans of the Galaxy>]


In [41]:
# UPDATE

movie_a = session.query(Movie).get(2)
movie_a.description = "Cool movie"
print(movie_a.id, movie_a.name, movie_a.description)
session.commit()

2 Matrix Revolutions Cool movie


In [42]:
movie_a = session.query(Movie).get(2)
print(movie_a.description)

Cool movie


In [43]:
# RENAME A MOVIE
qs = session.query(Movie).filter(Movie.name.contains("Matrix")).all()
for movie_obj in qs:
    movie_obj.name = "Matrix Revolutions"
session.commit()

In [44]:
qs = session.query(Movie).filter(Movie.name.contains("Matrix")).all()
qs

[<Movie name=Matrix Revolutions>,
 <Movie name=Matrix Revolutions>,
 <Movie name=Matrix Revolutions>]

In [61]:
# DELETE
movie_a = session.query(Movie).get(7)
print(movie_a)
session.delete(movie_a)
session.commit()

<Movie name=Lovecraft Country>


In [49]:
session.flush()

In [50]:
qs = session.query(Movie).all()
print(qs)

[<Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>, <Movie name=Interstellar>, <Movie name=Matrix Revolutions>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>, <Movie name=Interstellar>, <Movie name=Interstellar>, <Movie name=Matrix Revolutions>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>]


#### Hacer __ingenieria inversa__ con `sqlacodegen` para saber cual es el modelo(s) de una base de datos desde el archivo `.db`

Instalar el paquede `sqlacodegen`

In [ ]:
# https://pypi.org/project/sqlacodegen/

### SQLAlchemy & Pandas

In [57]:
!sqlacodegen sqlite:///app_test.db

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)


In [51]:
# SE COPIA EL MODELO PARA PODER OTRA BD CON OTRA SESION

In [52]:
engine = create_engine("sqlite:///app_test.db")
Session = sessionmaker(bind=engine)
session = Session()

In [53]:
# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class MovieApp2(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return f"<Movie name={self.name}>"

In [54]:
qs = session.query(MovieApp2).all()
print(qs)

[<Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>, <Movie name=Interstellar>, <Movie name=Matrix Revolutions>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>, <Movie name=Interstellar>, <Movie name=Interstellar>, <Movie name=Matrix Revolutions>, <Movie name=Lovecraft Country>, <Movie name=Guaridans of the Galaxy>]


In [55]:
# SABER QUE ES LO QUE CONTIENE EL QUERY SET ANTERIOR
for old_obj in qs:
    stored_data_dict = old_obj.__dict__
    print(stored_data_dict)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0dacf36f70>, 'name': 'Lovecraft Country', 'description': None, 'year': None, 'id': 3, 'genre': 'Sci-Fi'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0dacf36f40>, 'name': 'Guaridans of the Galaxy', 'description': None, 'year': None, 'id': 4, 'genre': None}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0dacf41040>, 'name': 'Interstellar', 'description': None, 'year': None, 'id': 5, 'genre': 'Sci-Fi'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0dacf410a0>, 'name': 'Matrix Revolutions', 'description': None, 'year': None, 'id': 6, 'genre': 'Sci-Fi'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0dacf41100>, 'name': 'Lovecraft Country', 'description': None, 'year': None, 'id': 7, 'genre': 'Sci-Fi'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0dacf41160>, 'name': 'Guaridans of the 

In [56]:
# COPIAR LOS DATOS DE UNA BASE DE DATOS A OTRA BD CON sqlacodegen
for old_obj in qs:
    movie_obj = Movie(name=old_obj.name, genre=old_obj.genre, year=old_obj.year)
    print(movie_obj.name)
    my_sess.add(movie_obj) # ES LA SESSION QUE ESTA ARRIBA CON LA BD DE LA TABLA Movies Y SE INSERTA A MoviesApp2
my_sess.commit()

Lovecraft Country
Guaridans of the Galaxy
Interstellar
Matrix Revolutions
Lovecraft Country
Guaridans of the Galaxy
Interstellar
Interstellar
Matrix Revolutions
Lovecraft Country
Guaridans of the Galaxy


In [ ]:
# HACER LO CON pandas

In [62]:
# MOSTRAR CON PANDAS UNA(ANTERIOR) BD DE SQLITE
old_engine = create_engine("sqlite:///app_test.db")
old_df = pd.read_sql_table("movies", old_engine)
old_df.head()

id                     name   genre description  year
0   3        Lovecraft Country  Sci-Fi        None  None
1   4  Guaridans of the Galaxy    None        None  None
2   5             Interstellar  Sci-Fi        None  None
3   6       Matrix Revolutions  Sci-Fi        None  None
4   8  Guaridans of the Galaxy    None        None  None

In [60]:
# MOSTRAR CON PANDAS UNA(NUEVA) BD DE SQLITE
current_engine = create_engine("sqlite:///app2.db")
current_df = pd.read_sql_table("movies", current_engine)
current_df.head()

id          name   genre description  year
0   1  Interstellar  Sci-Fi        None  None

In [63]:
# CONCATENAR LAS DOS BD A 20
final_df = pd.concat([current_df, old_df])
final_df.head(n=20)

id                     name   genre description  year
0    1             Interstellar  Sci-Fi        None  None
0    3        Lovecraft Country  Sci-Fi        None  None
1    4  Guaridans of the Galaxy    None        None  None
2    5             Interstellar  Sci-Fi        None  None
3    6       Matrix Revolutions  Sci-Fi        None  None
4    8  Guaridans of the Galaxy    None        None  None
5    9             Interstellar  Sci-Fi        None  None
6   10             Interstellar  Sci-Fi        None  None
7   11       Matrix Revolutions  Sci-Fi        None  None
8   12        Lovecraft Country  Sci-Fi        None  None
9   13  Guaridans of the Galaxy    None        None  None
10  14        Lovecraft Country  Sci-Fi        None  None
11  15  Guaridans of the Galaxy    None        None  None
12  16             Interstellar  Sci-Fi        None  None
13  17       Matrix Revolutions  Sci-Fi        None  None
14  18        Lovecraft Country  Sci-Fi        None  None
15  19  Guaridans of the Galaxy    None        None  None
16  20             Interstellar  Sci-Fi        None  None
17  21             Interstellar  Sci-Fi        None  None
18  22       Matrix Revolutions  Sci-Fi        None  None

In [64]:
# RE-ORGANIZAR EL INDEX DE LA CONCATENACIÓN DE LAS DOS BD A 20
final_df = final_df[['name', 'genre', 'description', 'year']]
final_df.reset_index(inplace=True, drop=True) # EYE
final_df.head(n=20)

name   genre description  year
0              Interstellar  Sci-Fi        None  None
1         Lovecraft Country  Sci-Fi        None  None
2   Guaridans of the Galaxy    None        None  None
3              Interstellar  Sci-Fi        None  None
4        Matrix Revolutions  Sci-Fi        None  None
5   Guaridans of the Galaxy    None        None  None
6              Interstellar  Sci-Fi        None  None
7              Interstellar  Sci-Fi        None  None
8        Matrix Revolutions  Sci-Fi        None  None
9         Lovecraft Country  Sci-Fi        None  None
10  Guaridans of the Galaxy    None        None  None
11        Lovecraft Country  Sci-Fi        None  None
12  Guaridans of the Galaxy    None        None  None
13             Interstellar  Sci-Fi        None  None
14       Matrix Revolutions  Sci-Fi        None  None
15        Lovecraft Country  Sci-Fi        None  None
16  Guaridans of the Galaxy    None        None  None
17             Interstellar  Sci-Fi        None  None
18             Interstellar  Sci-Fi        None  None
19       Matrix Revolutions  Sci-Fi        None  None

In [65]:
# AGREGAR UNA NUEVA TABLA A LA BD app2.db CON current_engine SI EXISTE SE REMPLASA
final_df.to_sql(
    "movies_2",
    current_engine,
    if_exists='replace',
    dtype = {
        "name": String,
        "genre": String,
        "year": Integer,
        "description": String,
    }
)

In [66]:
!sqlacodegen sqlite:///app2.db

# coding: utf-8
from sqlalchemy import BigInteger, Column, Integer, String, Table
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)


t_movies_2 = Table(
    'movies_2', metadata,
    Column('index', BigInteger, index=True),
    Column('name', String),
    Column('genre', String),
    Column('description', String),
    Column('year', Integer)
)


In [69]:
# CON sqlite3 HACER LA CONECCION Y LEER PANDAS Y SQL LA BD
import sqlite3

conx = sqlite3.connect("app_test.db")  # SE PUEDE DE ESTA FORMA
conx2 = create_engine("sqlite:///app_test.db")  # TAMBIEN SE PUEDE CON create_engine
df = pd.read_sql_query("SELECT * FROM movies", conx)
df.head()

id                     name   genre description  year
0   3        Lovecraft Country  Sci-Fi        None  None
1   4  Guaridans of the Galaxy    None        None  None
2   5             Interstellar  Sci-Fi        None  None
3   6       Matrix Revolutions  Sci-Fi        None  None
4   8  Guaridans of the Galaxy    None        None  None